# Да не посрамим землю русскую

In [64]:
import pandas as pd
import numpy as np

import pandas as pd
import numpy as np
import nltk
import keras
import tensorflow as tf
import matplotlib
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
import sklearn
import re
import string
from joblib import dump, load
import tensorflow.keras as keras
from keras.wrappers.scikit_learn import KerasClassifier
import gc

from sklearn.feature_extraction.text import CountVectorizer

In [65]:
train_data_set = pd.read_excel('data/dnn_class_predictions.xlsx', index_col = 0)
train_data_set.columns = list(map(str, train_data_set.columns))

prediction_data = pd.read_excel('data/test.xlsx', index_col = 0)

In [66]:
real_vars = ['Protein_(g)','Lipid_Tot_(g)','Carbohydrt_(g)']
label = 'label'
data = train_data_set[['Shrt_Desc','Protein_(g)','Lipid_Tot_(g)','Carbohydrt_(g)','label']]

In [67]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, r2_score,log_loss

In [42]:
food_names = train_data_set['Shrt_Desc'].copy()

def listToString(s):  
    str1 = ""  
    for ele in s:  
        str1 = str1 + ele + ' '
    return str1.lower()

food_names = food_names.map(lambda x : x.split(','))
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

#helpf = X_train

food_names = food_names.map(lambda x : [lemmatizer.lemmatize(i,  wordnet.VERB) for i in x])
food_names = food_names.map(lambda x : [word for word in x if word not in stop_words])
food_names = food_names.map(lambda x : [nltk.word_tokenize(i) for i in x] )
food_names = food_names.map(lambda x : [item for f in x for item in f])

table = str.maketrans('', '', string.punctuation)

food_names = food_names.map(lambda x : [i.translate(table) for i in x] )
food_names = food_names.map(lambda x : list(filter(None, x)))
food_names = food_names.map(lambda x : [re.sub(r'\d', '', i) for i in x])
food_names = food_names.map(lambda x : list(filter(None, x)))

count_vectorizer = CountVectorizer()
corpus = []

food_names.map(lambda x : corpus.append(listToString(x)))

bag_of_words = count_vectorizer.fit_transform(corpus)
feature_names = count_vectorizer.get_feature_names()
names_clf_train_x = pd.DataFrame(bag_of_words.toarray(), columns = feature_names)
names_clf_train_x

train_data_set = pd.concat([train_data_set, names_clf_train_x], axis = 1)

In [68]:
train, test, train_labels, test_labels = train_test_split(train_data_set.drop('label', axis = 1), train_data_set[label], test_size = 0.2, random_state = 42)
# train.to_excel('test_preds/clf3-trainset.xlsx')
# test.to_excel('test_preds/clf3-testset.xlsx')
# train_labels.to_csv('test_preds/clf3-train_labelsset.xlsx')
# test_labels.to_csv('test_preds/clf3-test_labelsset.xlsx')

# Random Forest

In [69]:
rf = RandomForestClassifier(n_estimators = 50, random_state = 42, max_depth=20)

rf_pipe = make_pipeline(ColumnSelector(cols=real_vars),
                        MinMaxScaler(),
                        rf)
rf_pipe.fit(train, train_labels)

Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=['Protein_(g)', 'Lipid_Tot_(g)',
                                      'Carbohydrt_(g)'],
                                drop_axis=False)),
                ('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=20, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=50,

In [70]:
res_pred=rf_pipe.predict(test)
res_pred

array([3, 1, 3, ..., 2, 3, 0], dtype=int64)

In [71]:
r2_score(test_labels,res_pred)

0.8691557618649854

In [80]:
pred_proba=rf_pipe.predict_proba(test)
train_proba=rf_pipe.predict_proba(train)
pd.DataFrame(train_proba).to_csv('test_preds/rf_train_pred.scv', index = False)
pd.DataFrame(pred_proba).to_csv('test_preds/rf_test_pred.scv', index = False)
print(train_proba.shape)
print(pred_proba.shape)


(6204, 5)
(1552, 5)


# Gradient Boosting

In [73]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(n_estimators=100, learning_rate=0.01,max_depth=8, random_state=4)

gb_pipe = make_pipeline(ColumnSelector(cols=real_vars),
                        MinMaxScaler(),
                        gb)
gb_pipe.fit(train,train_labels)

Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=['Protein_(g)', 'Lipid_Tot_(g)',
                                      'Carbohydrt_(g)'],
                                drop_axis=False)),
                ('minmaxscaler', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('gradientboostingclassifier',
                 GradientBoostingClassifier(ccp_alpha=0.0,
                                            criterion='friedman_mse', init=None,
                                            learning_rate=0.01, loss='deviance',
                                            max_depth=8, max_features=None,
                                            max_leaf_nodes=None,
                                            min_impurity_decrease=0.0,
                                            min_impurity_split=None,
                                            min_samples_leaf=1,
                                            min_samples_split=2,
      

In [74]:
gbp=gb_pipe.predict(test)

In [75]:
gb_proba=gb_pipe.predict_proba(test)
gb_train_proba=gb_pipe.predict_proba(train)
pd.DataFrame(gb_train_proba).to_csv('test_preds/gb_train_pred.scv', index = False)
pd.DataFrame(gb_proba).to_csv('test_preds/gb_test_pred.scv', index = False)
gb_proba

array([[0.06170717, 0.06043455, 0.03994117, 0.8311683 , 0.00674882],
       [0.0744772 , 0.7605725 , 0.06188676, 0.09486197, 0.00820157],
       [0.06883886, 0.06772047, 0.0444289 , 0.81148297, 0.0075288 ],
       ...,
       [0.08489497, 0.08332212, 0.71463582, 0.10785148, 0.00929562],
       [0.06631818, 0.06501806, 0.04298341, 0.81839953, 0.00728082],
       [0.77334193, 0.06931407, 0.06278445, 0.08710392, 0.00745564]])

In [76]:
gb_proba.mean()

0.2

In [77]:
r2_score(test_labels,gbp)

0.8488022137106498

# Make Predictions

In [ ]:
pred_res_rf = rf_pipe.predict_proba(prediction_data)
pred_res_gb = gb_pipe.predict_proba(prediction_data)

pd.DataFrame(pred_res_rf).to_csv('main-preds/rf_main_pred.scv', index = False)
pd.DataFrame(pred_res_gb).to_csv('main-preds/gb_main_pred.scv', index = False)

# DNN classifier

In [26]:
# import tensorflow.keras as keras
# from keras.wrappers.scikit_learn import KerasClassifier
# keras.backend.clear_session()

# def learning_rate_reduction():
#     return keras.callbacks.ReduceLROnPlateau(monitor='val_acc', 
#                                             patience=4, 
#                                             verbose=1, 
#                                             factor=0.4, 
#                                             min_lr=0.00002,
#                                             mode='max')
# def r2_keras(y_true, y_pred):
#     from keras import backend as K
#     SS_res =  K.sum(K.square(y_true - y_pred)) 
#     SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
#     return ( 1 - SS_res/(SS_tot + K.epsilon()) )

# mcp_save = keras.callbacks.ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_acc', verbose = 1, mode='max')

# def create_keras_model():
#     model = keras.models.Sequential([
#         keras.layers.BatchNormalization(input_shape = (3,)),
#         keras.layers.Dense(5, activation="relu"),
#         keras.layers.BatchNormalization(),
#         keras.layers.Dropout(0.2),
#         keras.layers.Dense(1, activation="relu"),
#         keras.layers.Dense(5, activation="softmax")])

#     keras.callbacks.LambdaCallback(on_epoch_end = lambda: model.load_weights('.mdl_wts.hdf5'))

#     model.compile(optimizer = keras.optimizers.Adam(),
#                   loss = "sparse_categorical_crossentropy",
#                   metrics = ['accuracy', r2_keras])
#     return model

# dnn_clf = KerasClassifier(build_fn = create_keras_model, verbose=1)

In [134]:
# dnn_clf.fit(train,
#           train_labels,
#           batch_size = 32,
#           epochs = 15,
#           validation_data=(test,test_labels),
#           callbacks=[learning_rate_reduction(), mcp_save])

# dnn_clf.fit(train,
#           train_labels,
#           batch_size = 64,
#           epochs = 15,
#           validation_data=(test,test_labels),
#           callbacks=[learning_rate_reduction(), mcp_save])

# dnn_clf.fit(train,
#           train_labels,
#           batch_size = 128,
#           epochs = 15,
#           validation_data=(test,test_labels),
#           callbacks=[learning_rate_reduction(), mcp_save])

# dnn_clf.fit(train,
#           train_labels,
#           batch_size = 256,
#           epochs = 15,
#           validation_data=(test,test_labels),
#           callbacks=[learning_rate_reduction(), mcp_save])

Train on 6204 samples, validate on 1552 samples
Epoch 1/15
6048/6204 [============================>.] - ETA: 0s - loss: 1.6450 - acc: 0.2682 - r2_keras: -11.3950
Epoch 00001: val_acc improved from -inf to 0.33312, saving model to .mdl_wts.hdf5
6204/6204 [==============================] - 2s 249us/sample - loss: 1.6428 - acc: 0.2663 - r2_keras: -11.3973 - val_loss: 1.5592 - val_acc: 0.3331 - val_r2_keras: -11.5389
Epoch 2/15
5920/6204 [===========================>..] - ETA: 0s - loss: 1.4206 - acc: 0.3301 - r2_keras: -11.4529
Epoch 00002: val_acc improved from 0.33312 to 0.51611, saving model to .mdl_wts.hdf5
6204/6204 [==============================] - 1s 168us/sample - loss: 1.4147 - acc: 0.3348 - r2_keras: -11.4377 - val_loss: 1.3479 - val_acc: 0.5161 - val_r2_keras: -11.5264
Epoch 3/15
6080/6204 [============================>.] - ETA: 0s - loss: 1.2421 - acc: 0.5331 - r2_keras: -11.5794
Epoch 00003: val_acc improved from 0.51611 to 0.55606, saving model to .mdl_wts.hdf5
6204/6204 [=

# Name Preprocessing

In [53]:
# import pandas as pd
# import numpy as np
# import nltk
# import keras
# import tensorflow as tf
# import matplotlib
# from nltk.stem import PorterStemmer, WordNetLemmatizer
# from nltk.corpus import wordnet
# from nltk.corpus import stopwords
# import sklearn
# import re
# import string
# from sklearn.feature_extraction.text import CountVectorizer

In [54]:
# nltk.download('wordnet')

In [58]:
# keras.backend.clear_session()

# def learning_rate_reduction():
#     return keras.callbacks.ReduceLROnPlateau(monitor='val_acc', 
#                                             patience=4, 
#                                             verbose=1, 
#                                             factor=0.4, 
#                                             min_lr=0.00002,
#                                             mode='max')
# def r2_keras(y_true, y_pred):
#     from keras import backend as K
#     SS_res =  K.sum(K.square(y_true - y_pred)) 
#     SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
#     return ( 1 - SS_res/(SS_tot + K.epsilon()) )

# mcp_save = keras.callbacks.ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_acc', verbose = 1, mode='max')

# def create_keras_mode_namel():
#     model1 = keras.models.Sequential([
#         keras.layers.Dense(256, input_shape= (3720,), activation='relu'),
#         keras.layers.Dropout(0,4),
#         keras.layers.Dense(128, activation='relu'),
#         keras.layers.Dropout(0,2),
#         keras.layers.Dense(64, activation='relu'),
#         keras.layers.Dense(5, activation='sigmoid')])

#     model1.compile(optimizer='adam',
#                    loss='sparse_categorical_crossentropy', 
#                    metrics=['accuracy'])
#     print(model1.summary())



# name_clf = KerasClassifier(build_fn=create_keras_mode_namel, verbose=1)

In [28]:
# print(train_corp.shape)
# print(test_corp.shape)
# print(train_labels_corp.shape)
# print(test_labels_corp.shape)

NameError: name 'train_corp' is not defined

In [59]:
name_clf.fit(train,
          train_labels,
          batch_size = 32,
          epochs = 15,
          validation_data=(test, test_labels),
          )

name_clf.fit(train,
          train_labels,
          batch_size = 64,
          epochs = 15,
          validation_data=(test, test_labels),
          )

 
name_clf.fit(train,
          train_labels,
          batch_size = 128,
          epochs = 15,
          validation_data=(test, test_labels),
          )

 
name_clf.fit(train,
          train_labels,
          batch_size = 256,
          epochs = 15,
          validation_data=(test, test_labels),
          )

TypeError: 'int' object is not iterable

In [177]:
accuracy_score(name_clf.predict(test_corp), test_labels_corp)

0.8634020618556701

In [ ]:
pd.DataSet(name_clf.predict(test)).to_scv('test_preds/name_clf_test_pred.scv', index = False)

In [126]:
len(list(train.columns[57:]))


3720

In [36]:
from mlxtend.classifier import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from mlxtend.feature_selection import ColumnSelector

name_clf_pipe = make_pipeline(ColumnSelector(cols=list(train.columns[57:])),
                              name_clf)

dnn_clf_pipe = make_pipeline(ColumnSelector(cols=real_vars),
                             dnn_clf)

estimators = [gb_pipe, rf_pipe,name_clf_pipe]

ensamble = StackingClassifier(classifiers = estimators, 
                              use_probas=True,
                              meta_classifier=LogisticRegression())


gc.collect()
# np.mean(n, axis =2)
#gb_pipe.fit(train, train_labels)
#(dnn_clf.predict_proba(test))
print(name_clf_pipe.fit(train, train_labels))
#print(r2_score(test_labels, ensamble.predict(test)))

TypeError: 'int' object is not iterable

In [88]:
p = pd.read_excel('data/test.xlsx')

pp = p[['Protein_(g)','Lipid_Tot_(g)','Carbohydrt_(g)']]

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

pp = scaler.fit_transform(pp)

pp = pd.concat([p, pd.DataFrame(dnn_clf.predict_proba(pp))], axis = 1)
pp
#pd.DataFrame(dnn_clf.predict(test))

,Shrt_Desc,Water_(g),Protein_(g),Lipid_Tot_(g),Ash_(g),Carbohydrt_(g),Fiber_TD_(g),Sugar_Tot_(g),Calcium_(mg),Iron_(mg),...,GmWt_1,GmWt_Desc1,GmWt_2,GmWt_Desc2,Refuse_Pct,0,1,2,3,4
0,SILK KEY LIME SOY YOGURT,77.59,2.35,1.18,1.24,17.65,0.6,12.35,176.0,0.64,...,170.0,1 container,NaN,NaN,0.0,0.000233,9.944793e-01,0.005251,1.841329e-05,1.900786e-05
1,"FAST FOODS,SALAD,VEG,TOSSED,WO/DRSNG",95.51,1.25,0.07,0.53,3.22,NaN,NaN,13.0,0.63,...,104.0,.75 cup,207.00,1.5 cup,0.0,0.000029,9.911026e-01,0.008828,2.828437e-05,1.246810e-05
2,"CUTTLEFISH,MXD SP,CKD,MOIST HEAT",61.12,32.48,1.40,3.36,1.64,0.0,NaN,180.0,10.84,...,85.0,3 oz,NaN,NaN,0.0,0.001141,1.549279e-03,0.994182,2.989389e-03,1.383081e-04
3,"GROUND TURKEY,93% LN,7% FAT,RAW",72.63,18.73,8.34,0.92,0.00,0.0,0.00,21.0,1.17,...,28.0,1 oz,453.00,1 lb,0.0,0.000012,6.825158e-04,0.999256,4.624214e-05,3.133727e-06
4,"TURKEY,RTL PARTS,ENHANCED,BREAST,MEAT ONLY,CKD...",69.99,27.94,2.08,1.40,0.00,0.0,0.00,15.0,0.59,...,85.0,3 oz,852.00,1 breast,24.0,0.000066,1.145381e-03,0.998463,3.048181e-04,2.030744e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,"CEREALS,QUAKER,OATMEAL,REAL MEDLEYS,SMMR BERRY...",7.57,11.31,4.40,4.14,72.58,8.5,20.43,124.0,3.13,...,70.0,"1 package, (1 NLEA serving)",NaN,NaN,0.0,0.999966,1.095290e-07,0.000033,5.635981e-07,2.858141e-07
858,"VEAL,SHLDR,BLADE,LN,CKD,BRSD",59.24,32.66,6.48,1.24,0.00,0.0,0.00,40.0,1.47,...,85.0,3 oz,174.00,"1 piece, cooked, excluding refuse (yield from ...",39.0,0.000105,1.262476e-04,0.998566,1.182745e-03,1.946694e-05
859,"ONIONS,CKD,BLD,DRND,W/SALT",87.86,1.36,0.19,1.03,9.56,1.4,4.73,22.0,0.24,...,210.0,1 cup,15.00,"1 tbsp, chopped",0.0,0.000039,9.984369e-01,0.001505,1.172247e-05,7.367324e-06
860,"KEEBLER,FUDGE SHOPPE,JUMBO FUDGE STKS,MINT",1.70,2.60,27.20,NaN,67.60,1.8,50.10,NaN,1.60,...,31.0,1 cookie,NaN,NaN,0.0,0.999986,7.587086e-09,0.000011,2.750408e-06,6.862309e-07


# MainModel2

In [3]:
from catboost import Pool, CatBoostRegressor

In [10]:
data = pd.read_excel('data/dnn1_class_predictions.xlsx')
label = data['Energ_Kcal']
train_data1 = pd.DataFrame()
train_data1['Lip'] = data['Lipid_Tot_(g)']/100
train_data1['Prot'] = data['Protein_(g)']/88
train_data1['Carbohydrt'] = data['Carbohydrt_(g)']/100
train_data1 =  train_data1.join(data.iloc[:, 54:69])
from sklearn import model_selection 
X_train, X_test, Y_train,  Y_test = sklearn.model_selection.train_test_split(train_data1, label, random_state=1)

In [11]:
best_model = CatBoostRegressor(
   iterations=200000,
   learning_rate = 0.07521709965938336,
   random_seed=63,
   loss_function="MAE",
)

In [12]:
train_pool = Pool(X_train,
                  Y_train)
test_pool = Pool(X_test,
                 Y_test) 
best_model.fit(train_pool, eval_set=(test_pool), logging_level='Verbose')

0:	learn: 119.7805858	test: 129.9761673	best: 129.9761673 (0)	total: 59.1ms	remaining: 3h 16m 57s
1:	learn: 110.9430530	test: 120.5130588	best: 120.5130588 (1)	total: 61.3ms	remaining: 1h 42m 11s
2:	learn: 103.2495258	test: 112.3749352	best: 112.3749352 (2)	total: 64.2ms	remaining: 1h 11m 22s
3:	learn: 96.0259332	test: 104.5588484	best: 104.5588484 (3)	total: 66.7ms	remaining: 55m 35s
4:	learn: 89.0570069	test: 97.1821788	best: 97.1821788 (4)	total: 69.1ms	remaining: 46m 2s
5:	learn: 82.6537015	test: 90.3579391	best: 90.3579391 (5)	total: 71.4ms	remaining: 39m 40s
6:	learn: 76.6099933	test: 83.8724841	best: 83.8724841 (6)	total: 73.7ms	remaining: 35m 5s
7:	learn: 71.2506559	test: 78.1560181	best: 78.1560181 (7)	total: 76.1ms	remaining: 31m 42s
8:	learn: 66.3820471	test: 72.9097669	best: 72.9097669 (8)	total: 78.9ms	remaining: 29m 12s
9:	learn: 62.3765014	test: 68.5717827	best: 68.5717827 (9)	total: 81.3ms	remaining: 27m 6s
10:	learn: 58.5165278	test: 64.4336133	best: 64.4336133 (10)	to

KeyboardInterrupt: 

In [14]:
simple=pd.read_excel('data/simple.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: 'data/simple.xlsx'